In [1]:
# DB → cmds 파일 생성 (댓글용 - 재사용 가능 버전)
import json
import pymysql

# ==================== DB 설정 ====================
USER = "root"
PASS = "1234"
HOST = "127.0.0.1"
PORT = 3306
DB = "crawling"

# ==================== 설정 영역 (여기를 수정하세요) ====================
# 게시판 타입 (필터링용)
BOARD_TYPE = "free"  # ✨ 추가: hredu, cp, sales 등

# DB 테이블명 (이제 고정)
TABLE_NAME = "board_comments"  # ✨ 통합 테이블명

# 출력 파일명
OUTPUT_FILE = f"comments_{BOARD_TYPE}.cmds"  # ✨ 자동으로 타입 반영

# ======================================================================

def generate_comments_cmds_file(table_name, board_type, output_file):
    """DB에서 특정 type의 댓글 데이터를 읽어 cmds 파일 생성"""
    
    print(f"📂 DB 연결 중...")
    print(f"📂 테이블: {table_name}")
    print(f"📂 게시판 타입: {board_type}")  # ✨ 추가
    print(f"📂 출력 파일: {output_file}\n")
    
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASS,
        database=DB,
        port=PORT,
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. DB에서 특정 type의 댓글 데이터만 읽기
            query = f"SELECT * FROM {table_name} WHERE type = %s"  # ✨ WHERE 조건 추가
            cursor.execute(query, (board_type,))
            comments = cursor.fetchall()
            
            total_count = len(comments)
            print(f"✅ 총 {total_count}개의 댓글 발견\n")
            
            # 2. cmds 파일 생성
            with open(output_file, 'w', encoding='utf-8') as f:
                for idx, comment in enumerate(comments, 1):
                    try:
                        # cmds 형식에 맞는 JSON 객체 생성
                        comment_data = {
                            "sourceId": comment.get('sourceId', ''),
                            "sourcePostId": comment.get('sourcePostId', ''),
                            "createdAt": comment.get('createdAt', ''),
                            "writer": {
                                "emailId": comment.get('writer_emailId', '')
                            },
                            "message": comment.get('message', ''),
                            "attaches": comment.get('attaches', '')
                        }
                        
                        # addPostComment{JSON} 형식으로 작성
                        line = 'addPostComment' + json.dumps(comment_data, ensure_ascii=False)
                        f.write(line + '\n')
                        
                        if idx % 100 == 0:
                            print(f"   처리 중... {idx}/{total_count}")
                        
                    except Exception as e:
                        print(f"❌ [Comment {idx}] 처리 오류: {e}")
                        continue
            
            print(f"\n{'='*60}")
            print(f"✅ cmds 파일 생성 완료!")
            print(f"   - 게시판 타입: {board_type}")  # ✨ 추가
            print(f"   - 총 댓글: {total_count}개")
            print(f"   - 파일 위치: {output_file}")
            print(f"{'='*60}")
    
    finally:
        connection.close()
        print("\n✅ DB 연결 종료")

# 실행
if __name__ == "__main__":
    generate_comments_cmds_file(TABLE_NAME, BOARD_TYPE, OUTPUT_FILE)  # ✨ board_type 인자 추가

📂 DB 연결 중...
📂 테이블: board_comments
📂 게시판 타입: free
📂 출력 파일: comments_free.cmds

✅ 총 415개의 댓글 발견

   처리 중... 100/415
   처리 중... 200/415
   처리 중... 300/415
   처리 중... 400/415

✅ cmds 파일 생성 완료!
   - 게시판 타입: free
   - 총 댓글: 415개
   - 파일 위치: comments_free.cmds

✅ DB 연결 종료
